# `Python` 基础（1）

## 装饰器

在学习`FastAPI` 中，相信你肯定看到`@app.get`，如果你是`Python`初学者，你肯定会比较好奇，这个定义 `HTTP` 路由的方式是如何实现的，如果我自己要实现这个东西，应该如何做。

其实上述实现原理基于`Python`的装饰器语法，通过将被装饰的函数与特定的 `HTTP` 方法（`GET`）、路径（`path`）及相关配置绑定。

然后，你又会好奇，装饰器到底是什么东西？**装饰器其实就是将函数传递给函数**。

先看一个例子：

In [8]:
import time


def decorator(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"程序{func.__name__}运行时间：{end-start} 秒。")
    return wrapper

@decorator
def square(x):
    return x * x

In [9]:
square(10)

程序square运行时间：1.430511474609375e-06 秒。


现在有一个问题，我们使用`fastapi`时：

```python
@app.get('/home')
async def root():
    pass
```

这个注解是有参数的，怎么让decorator带上参数呢？直接加一个参数看看：

In [10]:
import time


def decorator(func, name):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"程序{func.__name__}运行时间：{end-start} 秒。")
    return wrapper

@decorator(name="ab")
def square(x):
    return x * x

TypeError: decorator() missing 1 required positional argument: 'func'

结果是不行的，方法就是再套一层：

In [11]:
import functools

def get(url):
    def decorator(func):
        # 下面这个注解只是让被装饰器的名字不变化
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            print(f"执行URL：{url}")
            result = func(*args, **kwargs)
            end = time.time()
            print(f"程序{func.__name__}运行时间：{end-start} 秒。")
        return wrapper
    return decorator

@get("www.example.com")
def square(x):
    return x * x

In [13]:
square(5)

执行URL：www.example.com
程序square运行时间：5.650520324707031e-05 秒。



现在是最后一个问题了，`app.get('/home')`中的`app`要根据不同对象名创建怎么做？

其实这个很简单，只需要这个装饰器在一个类里面就可以了。

现在让我们做一个假的`FastAPI`:

In [14]:

class FastAPI():
    def __init__(self):
        self.routers = {}

    def get(self, path:str, **kwargs):
        def decorator(func):
            self.routers[path] = {
                "method": ["GET"],
                "func": func,
                **kwargs
            }
            return func
        return decorator

In [ ]:
api = FastAPI()

@api.get('/hello', description='this is hello world')
def hello(name: str):
    return f"Hello, {name}"

api.routers

<function __main__.hello(name: str)>

到此，装饰器就已经讲完了，事实上装饰器非常简单。

当我们调用上面的装饰器来装饰函数时：

等同于调用：

```python
hello = api.get('/hello', description='this is hello world')(hello)
```

最后看一下`Python`内置的装饰器`property`。

### `property`属性`CRUD`装饰器

作用是在不改变类的外部调用方式的前提下，为类的属性的访问、修改和删除添加对应逻辑。

其应用场景包括：

- 数据验证：验证数据是否正确
- 计算属性：内部多个属性存在依赖可以自动计算
- 权限控制：隐藏内部实现，暴露必要操作

In [3]:
class Person:
    def __init__(self, age):
        self.age = age

p = Person(20)
print(p.age)  # 直接访问属性
p.age = -10    # 直接修改属性
p.age

20


-10

In [ ]:
class Person:
    def __init__(self, age):
        self._age = age

    # 用 @property 装饰，将方法转为属性
    @property
    def age(self):
        return self._age

p = Person(20)
print(p.age)  # 像访问属性一样调用，输出：20
# 只读属性，不能直接修改
p.age = 21

20


AttributeError: property 'age' of 'Person' object has no setter

In [ ]:
class Person:
    def __init__(self, age):
        self._age = age  # 存储实际值

    @property
    def age(self):
        return self._age  # 读取逻辑

    # 允许修改 age，并添加验证
    @age.setter
    def age(self, value):
        if not isinstance(value, int):
            raise TypeError("年龄必须是整数")
        if value < 0 or value > 150:
            raise ValueError("年龄必须在 0-150 之间")
        self._age = value  # 通过验证后才更新

    @age.deleter
    def age(self):
        # 删除之后 age 不可访问
        del self._age


p = Person(20)
print(p.age)  # 20

p.age = 25    # 合法修改，成功
print(p.age)  # 25

p.age = -5    # 不合法修改

# del p.age

20
25


ValueError: 年龄必须在 0-150 之间

## `__init__.py`

相信很多人都知道，导入`python`项目的的模块（文件），实际上就是执行这个文件里面的内容，但是如果导入一个`package`（目录），你就会发现啥都不会执行。

如果你在`package`（目录）下增加一个`__init__.py`，导入该`package`时，就会执行`__init__.py`文件。所以`__init__.py`就是做一些包的信息、初始化工作、`package`下需要导出的符号。

```python
# 一个 __init__.py 示范 假设包名是 yourPackage

# 包的版本及作者
__version__ = ' 1.0.1'
__author__ = 'Bob'

# 假设当前包下的文件为 youModuleName.py youModuleName2.py
from . import youModuleName  # 相对导入，
from .youModuleName import module_symbol
from .youModuleName2 import module_symbol_2

# from yourPackage import *
# 上述的导入就是导入 __all__ 这里面的内容
__all__ = ['module_symbol', 'module_symbol_2']
```

同一个`package`下，要使用**相对导入**，这是因为`Python`的导入路径是系统`Path`加上当前工作目录。而相对导入是从当前`package`作为搜索目录，其中：

- `.`: 当前目录
- `..`: 上级目录

> 注意：
>
> 目录后面不要跟`/`，如导入上级sub目录下的moduleA
> 
> ```python
> from ..sub import moduleA
> ```
> 



## 枚举

`python`的枚举使用起来非常简单，但是是特定场景下用起来和`rust`一样爽。

In [11]:
from enum import Enum, IntEnum, StrEnum

class Color(IntEnum):
    RED = 1
    GREEN = 2
    YELLOW = 3

    @classmethod
    def print_all(cls):
        print('---------------all-------------')
        for c in cls:
            print(c.name, c.value)

color = Color.RED
print(color)
color.print_all()

1
---------------all-------------
RED 1
GREEN 2
YELLOW 3


In [17]:

class HttpStatus(Enum):
    def __new__(cls, code: int, msg: str):
        member = object.__new__(cls)
        member._value_ = (code, msg)
        return member

    OK = 200, 'OK'
    NOT_FOUND = 404, 'NOT_FOUND'

print(HttpStatus.OK.name, HttpStatus.OK.value)

status = HttpStatus.OK
print(status)
# 和上面的方式是等价的
status = HttpStatus(200, 'OK')
print(status)

# 没在枚举中定义的值都会抛出异常
status = HttpStatus(100, 'Continue')
print(status)

OK (200, 'OK')
HttpStatus.OK
HttpStatus.OK


ValueError: (100, 'Continue') is not a valid HttpStatus